In [1]:
# noexport

!typech efficacy_utils.ipynb

[NbConvertApp] Converting notebook efficacy_utils.ipynb to python
retention_utils.py:448: error: Name 'get_ipython' is not defined
retention_utils.py:511: error: Name 'get_ipython' is not defined
efficacy_utils.py:40: error: Name 'get_condition_to_installs_for_random_assignment_abtest' already defined on line 27
efficacy_utils.py:163: error: Name 'get_ipython' is not defined
efficacy_utils.py:180: error: Name 'get_ipython' is not defined
efficacy_utils.py:184: error: Name 'get_ipython' is not defined
efficacy_utils.py:188: error: Name 'get_ipython' is not defined
efficacy_utils.py:192: error: Name 'get_ipython' is not defined
Found 8 errors in 2 files (checked 1 source file)


In [2]:
from retention_utils import *

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


<IPython.core.display.Javascript object>

R[write to console]: Error in withVisible({ : 找不到对象'plot_attrition_df'




Error in withVisible({ : 找不到对象'plot_attrition_df'


In [3]:
import jsonmemo as jsonmemo_module
jsonmemo_module.set_lowmem(True)

In [4]:
#installs_with_experiment_vars = get_installs_with_experiment_vars()

In [5]:
#print(installs_with_experiment_vars[0])

In [6]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen'
  groups = ['survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [7]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen_and_choose_difficulty_frequency'
  groups = ['survey', 'nodefault_forcedchoice_userchoice', 'survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  #condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  condition_to_installs = get_conditions_to_install_list_in_abtest(abtest_name, groups)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [8]:

#for install in installs_with_experiment_vars:
@msgpackmemo1arg
def get_domain_to_intervention_to_session_lengths_for_install(install):
  seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
  
  domain_to_session_to_intervention = {}
  for x in interventions_active_for_domain_and_session:
    if x.get('val') is None:
      continue
    interventions_active = json.loads(x['val'])
    if len(interventions_active) == 0:
      continue
    intervention_name = interventions_active[0]
    domain = x['key']
    session_id = x['key2']
    if domain not in domain_to_session_to_intervention:
      domain_to_session_to_intervention[domain] = {}
    domain_to_session_to_intervention[domain][session_id] = intervention_name
  
  domain_to_intervention_to_session_lengths = {}
  for x in seconds_on_domain_per_session:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_session')
      print(x)
      continue
    domain = x['key']
    session_id = x['key2']
    time_spent = x['val']
    if domain not in domain_to_session_to_intervention:
      continue
    intervention_name = domain_to_session_to_intervention[domain].get(session_id)
    if intervention_name is None:
      continue
    if domain not in domain_to_intervention_to_session_lengths:
      domain_to_intervention_to_session_lengths[domain] = {}
    if intervention_name not in domain_to_intervention_to_session_lengths[domain]:
      domain_to_intervention_to_session_lengths[domain][intervention_name] = []
    domain_to_intervention_to_session_lengths[domain][intervention_name].append(time_spent)
  return domain_to_intervention_to_session_lengths
  

In [9]:
# noexport

for x in get_collection_names():
  if 'per_day' in x and 'visits' not in x:
    print(x)
    break

edceda6041ef668213a96ba5_synced:seconds_on_domain_per_day


In [10]:
@msgpackmemo1arg
def get_epoch_to_domain_to_time_spent(install):
  seconds_on_domain_per_day = get_collection_for_install(install, 'synced:seconds_on_domain_per_day')
  output = {}
  for x in seconds_on_domain_per_day:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_day')
      print(x)
      continue
    domain = x['key']
    epoch = x['key2']
    seconds = x['val']
    if epoch not in output:
      output[epoch] = {}
    output[epoch][domain] = seconds
  return output

In [11]:
# noexport

condition_to_installs = get_condition_to_installs_for_random_assignment_abtest()


In [12]:
# noexport

for condition,installs in condition_to_installs.items():
  for install in installs:
    get_epoch_to_domain_to_time_spent(install)
    #print(sorted(get_epoch_to_domain_to_time_spent(install).keys()))
    break

In [13]:
import math

def make_domain_to_daily_time_dataframe():
  output = []
  for condition,installs in condition_to_installs.items():
    #condition_to_lengths[condition] = []
    for install in installs:
      for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
#         total_time_spent = sum(domain_to_time_spent.values())
#         output.append({
#           'user': install,
#           'epoch': epoch,
#           'time': total_time_spent,
#           'logtime': math.log(total_time_spent),
#           'condition': condition,
#         })
        for domain,total_time_spent in  domain_to_time_spent.items():
          output.append({
            'domain': domain,
            'user': install,
            'epoch': epoch,
            'time': total_time_spent,
            'logtime': math.log(total_time_spent),
            'condition': condition,
          })
  return to_dataframe(output)

In [14]:
# noexport

df = make_domain_to_daily_time_dataframe()

In [15]:
#print(df)

In [16]:
# noexport

%Rpush df

In [17]:
#%%R
#install.library

In [18]:
%%R

library(lme4)
#library(sjPlot)
library(lmerTest)
library(stargazer)


R[write to console]: 载入需要的程辑包：Matrix

R[write to console]: Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

R[write to console]: 
载入程辑包：‘lmerTest’


R[write to console]: The following object is masked from ‘package:lme4’:

    lmer


R[write to console]: The following object is masked from ‘package:stats’:

    step


R[write to console]: 
Please cite as: 


R[write to console]:  Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.

R[write to console]:  R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 




In [19]:
# %%R

# df$user <- as.factor(df$user)
# #df$domain <- as.factor(df$domain)
# df$condition <- as.factor(df$condition)
# df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard"))
# df$epoch <- as.factor(df$epoch)
# df$logtime <- as.numeric(df$logtime)
# df$time <- as.numeric(df$time)
# summary(df)



In [20]:
%%R

df$user <- as.factor(df$user)
df$domain <- as.factor(df$domain)
df$condition <- as.factor(df$condition)
df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard", "survey", "nodefault_forcedchoice_userchoice"))
df$epoch <- as.factor(df$epoch)
df$logtime <- as.numeric(df$logtime)
df$time <- as.numeric(df$time)
summary(df)




[[1]]
[1] "www.google.com  : 239777  "

[[2]]
[1] "www.youtube.com : 168371  "

[[3]]
[1] "mail.google.com : 154988  "

[[4]]
[1] "www.facebook.com: 112296  "

[[5]]
[1] "docs.google.com :  91410  "

[[6]]
[1] "drive.google.com:  62719  "

[[7]]
[1] "(Other)         :6518902  "

[[8]]
[1] "f9329b2116e3bd031cca0519:  24853  "

[[9]]
[1] "1e42870bb128918b18137b18:  22344  "

[[10]]
[1] "23b9ce6a12de923653ac786d:  22298  "

[[11]]
[1] "dd83217732fe6ba08644289a:  19079  "

[[12]]
[1] "d37967a410f11ae1d3d85f29:  18360  "

[[13]]
[1] "fdf4e9c44628fd753700ce44:  17516  "

[[14]]
[1] "(Other)                 :7224013  "

[[15]]
[1] "1600   :  28588  "

[[16]]
[1] "1578   :  28551  "

[[17]]
[1] "1573   :  28178  "

[[18]]
[1] "1607   :  27964  "

[[19]]
[1] "1565   :  27917  "

[[20]]
[1] "1567   :  27901  "

[[21]]
[1] "(Other):7179364  "

[[22]]
[1] "Min.   :    1.0  "

[[23]]
[1] "1st Qu.:   14.0  "

[[24]]
[1] "Median :   55.0  "

[[25]]
[1] "Mean   :  375.8  "

[[26]]
[1] "3rd Qu.:  201.0

In [21]:
%%R

#dfsub <- df[df$domain == 'www.youtube.com', ]
#show(summary(dfsub))
#show(summary(subset(df, domain='www.youtube.com')))
#dfsub <- subset(df, domain='www.youtube.com')
#show(summary(dfsub))

NULL


In [22]:
%%R

dfsub <- df
show(summary(dfsub))
#results <- lmer(logtime ~ condition + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user), data = dfsub)
results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
show(results)
show(summary(results))
class(results) <- "lmerMod"
stargazer(results)

              domain                              user        
 www.google.com  : 239777   f9329b2116e3bd031cca0519:  24853  
 www.youtube.com : 168371   1e42870bb128918b18137b18:  22344  
 mail.google.com : 154988   23b9ce6a12de923653ac786d:  22298  
 www.facebook.com: 112296   dd83217732fe6ba08644289a:  19079  
 docs.google.com :  91410   d37967a410f11ae1d3d85f29:  18360  
 drive.google.com:  62719   fdf4e9c44628fd753700ce44:  17516  
 (Other)         :6518902   (Other)                 :7224013  
     epoch              time            logtime      
 1600   :  28588   Min.   :    1.0   Min.   : 0.000  
 1578   :  28551   1st Qu.:   14.0   1st Qu.: 2.639  
 1573   :  28178   Median :   55.0   Median : 4.007  
 1607   :  27964   Mean   :  375.8   Mean   : 4.008  
 1565   :  27917   3rd Qu.:  201.0   3rd Qu.: 5.303  
 1567   :  27901   Max.   :89087.0   Max.   :11.397  
 (Other):7179364                                     
                             condition      
 survey_nochoice_no

In [23]:
%%R

dfsub <- df[df$domain == 'www.facebook.com', ]
show(summary(dfsub))
#results <- lmer(logtime ~ condition + (1|user), data = df)
results <- lmer(logtime ~ condition + (1|user), data = dfsub)
#results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
show(results)
show(summary(results))
class(results) <- "lmerMod"
stargazer(results)

                                          domain      
 www.facebook.com                            :112296  
 _.shoprintee.com                            :     0  
 _354ee77b690e05ac66854a6c6d9f5ad6.testera.ir:     0  
 _antonia.gr                                 :     0  
 _dmarc.adoreme.com                          :     0  
 _dnslink.your.website                       :     0  
 (Other)                                     :     0  
                       user            epoch             time      
 3d8f23613f3849ed266ace58:   418   1592   :   430   Min.   :    1  
 f9329b2116e3bd031cca0519:   387   1573   :   429   1st Qu.:  116  
 e02d077981cded46b3146e43:   366   1609   :   426   Median :  457  
 319241bc0a9dee7ccacbc105:   339   1587   :   423   Mean   : 1158  
 a7311fce31d8d92e37288573:   337   1599   :   422   3rd Qu.: 1355  
 269aa5d6e571d088ee3ad433:   325   1608   :   422   Max.   :88216  
 (Other)                 :110124   (Other):109744                  
    logtime     

In [24]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      condition_to_lengths[condition].append(total_time_spent)
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [25]:
# noexport
# bad


condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for domain,intervention_to_session_lengths in get_domain_to_intervention_to_session_lengths_for_install(install).items():
      if domain != 'www.youtube.com':
        continue
      for intervention,session_lengths in intervention_to_session_lengths.items():
        for session_length in session_lengths:
          condition_to_lengths[condition].append(session_length)


In [26]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    all_lengths_for_user = []
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      all_lengths_for_user.append(total_time_spent)
    if len(all_lengths_for_user) == 0:
      continue
    condition_to_lengths[condition].append(np.median(all_lengths_for_user))
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [27]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      if 'www.facebook.com' not in domain_to_time_spent:
        continue
      time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      condition_to_lengths[condition].append(time_on_facebook)

In [28]:
# noexport
from scipy.stats import ttest_ind

condition_to_mlengths = {}
for condition,session_lengths in condition_to_lengths.items():
  print(condition, np.mean(session_lengths))
  condition_to_mlengths[condition] = np.mean(session_lengths) / 60

#print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_nothing']))
print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_easy']))
#print(ttest_ind(condition_to_lengths['survey_nochoice_easy'], condition_to_lengths['survey_nochoice_medium']))


nodefault_forcedchoice_userchoice 945.6508664356172
survey 1146.7481527093596
survey_nochoice_hard 1152.458648973408
survey_nochoice_easy 1376.3333166508182
survey_nochoice_nothing 1050.2378763353836
survey_nochoice_medium 1239.4932259057784
Ttest_indResult(statistic=-9.386884766792647, pvalue=6.501266012485611e-21)


In [29]:
# noexport

from plot_utils import *


In [30]:
# noexport

#from plot_utils import *
plot_dict_as_bar(condition_to_mlengths, title='Total minutes spent daily on sites', xlabel='Difficulty', ylabel='Minutes spent daily on sites', remap_labels={'survey_nochoice_nothing': 'Nothing', 'survey_nochoice_easy': 'Easy', 'survey_nochoice_medium': 'Medium', 'survey_nochoice_hard': 'Hard', 'survey': 'Experience Sampling', 'nodefault_forcedchoice_userchoice': 'Onboarding Choice'})

In [31]:
# for install in installs_with_experiment_vars:
#   #seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
#   #interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
#   res = get_domain_to_intervention_to_session_lengths_for_install(install)
#   #if len(res) > 0:
#   #  print(res)
#   #  break